In [56]:
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype
from Evaluation.Evaluator import EvaluatorHoldout
import scipy.sparse as sps
from collections import Counter
from tqdm import tqdm
# imports for .env usage
import os
from dotenv import load_dotenv
load_dotenv()

True

In [57]:
def dataframe_to_csr(dataframe,row_name,col_name,cell_name):
        """This method converts a dataframe object into a csr

        Args:
            dataframe (dataframe)
            row_name (str): For example, "UserID"
            col_name (str): For example, "ItemID"
            cell_name (str): For example, "Data"
        Returns:
            csr
        """
        rows = dataframe[row_name].unique()
        columns = dataframe[col_name].unique()

        shape = (len(rows), len(columns))

        # Create indices for users and items
        row_cat = CategoricalDtype(categories=sorted(rows), ordered=True)
        col_cat = CategoricalDtype(categories=sorted(columns), ordered=True)
        row_index = dataframe[row_name].astype(row_cat).cat.codes
        col_index = dataframe[col_name].astype(col_cat).cat.codes

        # Conversion via COO matrix
        coo = sps.coo_matrix(
            (dataframe[cell_name], (row_index.values, col_index.values)), shape=shape)
        csr = coo.tocsr()
        return csr

In [116]:
interactions_and_impressions = pd.read_csv(filepath_or_buffer=os.getenv('INTERACTIONS_AND_IMPRESSIONS_PATH'),
                                            sep=',',
                                            names=[
    'UserID', 'ItemID', 'Impressions', 'Data'],
    header=0,
    dtype={'UserID': np.int32, 'ItemID': np.int32, 'Impressions': np.object0, 'Data': np.int32})
interactions = interactions_and_impressions.drop(['Impressions'], axis=1)
items = interactions['ItemID'].unique()
data = interactions['Data'].unique()
users = interactions['UserID'].unique()
print(items.shape)
print(data.shape)
print(users.shape)
interactions = interactions.replace({'Data': {0: 1}})
interactions = interactions.drop_duplicates(keep='first')
urm = dataframe_to_csr(interactions,'UserID','ItemID','Data')
interactions

(5826506,)
(5826506,)
(5826506,)


,UserID,ItemID,Data
0,0,11,1
1,0,21,1
13,0,22,1
28,0,24,1
29,0,44,1
...,...,...,...
5826501,41628,20448,1
5826502,41628,20896,1
5826503,41628,21506,1
5826504,41628,22882,1


In [59]:
print(interactions.ItemID.unique().size)
print(interactions.UserID.unique().size)

24507
41629


In [60]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(urm, train_percentage = 0.5)

In [61]:
print(URM_train.get_shape)

<bound method spmatrix.get_shape of <41629x24507 sparse matrix of type '<class 'numpy.float64'>'
	with 777320 stored elements in Compressed Sparse Row format>>


In [62]:
data_icm_type = pd.read_csv(filepath_or_buffer=os.getenv('DATA_ICM_TYPE_PATH'),
                            sep=',',
                            names=[
    'item_id', 'feature_id', 'data'],
    header=0,
    dtype={'item_id': np.int32, 'feature_id': np.int32, 'data': np.int32})

features = data_icm_type['feature_id'].unique()
items = data_icm_type['item_id'].unique()
shape = (len(items), len(features))


# Create indices for users and items
features_cat = CategoricalDtype(categories=sorted(features), ordered=True)
item_cat = CategoricalDtype(categories=sorted(items), ordered=True)
features_index = data_icm_type["feature_id"].astype(features_cat).cat.codes
item_index = data_icm_type["item_id"].astype(item_cat).cat.codes
coo = sps.coo_matrix(
    (data_icm_type["data"], (item_index.values, features_index.values)), shape=shape)
ICM = coo.tocsr()


In [63]:
print(ICM)

  (0, 0)	1
  (1, 2)	1
  (2, 3)	1
  (3, 0)	1
  (4, 2)	1
  (5, 1)	1
  (6, 0)	1
  (7, 2)	1
  (8, 0)	1
  (9, 0)	1
  (10, 0)	1
  (11, 0)	1
  (12, 2)	1
  (13, 0)	1
  (14, 0)	1
  (15, 2)	1
  (16, 0)	1
  (17, 0)	1
  (18, 3)	1
  (19, 3)	1
  (20, 1)	1
  (21, 2)	1
  (22, 0)	1
  (23, 0)	1
  (24, 3)	1
  :	:
  (23066, 3)	1
  (23067, 2)	1
  (23068, 0)	1
  (23069, 2)	1
  (23070, 2)	1
  (23071, 0)	1
  (23072, 0)	1
  (23073, 0)	1
  (23074, 0)	1
  (23075, 3)	1
  (23076, 0)	1
  (23077, 0)	1
  (23078, 3)	1
  (23079, 0)	1
  (23080, 2)	1
  (23081, 0)	1
  (23082, 2)	1
  (23083, 3)	1
  (23084, 0)	1
  (23085, 3)	1
  (23086, 0)	1
  (23087, 1)	1
  (23088, 0)	1
  (23089, 0)	1
  (23090, 3)	1


In [64]:
diff = np.setdiff1d(interactions['ItemID'].unique(), data_icm_type['item_id'].unique())
print(diff)

[    9    14    23 ... 24486 24492 24499]


In [65]:
print(diff.size)

4877


In [66]:
diff_1 = np.setdiff1d(data_icm_type['item_id'].unique(), interactions['ItemID'].unique())
print(diff_1)

[24507 24508 24509 ... 27965 27966 27967]


In [67]:
print(diff_1.size)

3461


In [68]:
from Data_Handler.DataReader import DataReader
dataReader = DataReader()

In [119]:
pad_items_ids = np.setdiff1d(dataReader.load_augmented_binary_urm_df()['ItemID'].unique(), dataReader.load_icm_df()['item_id'].unique())
feature_ids = dataReader.load_icm_df()['feature_id'].unique()
col = []
for feature_id in feature_ids:
    for item_id in range(len(pad_items_ids)):
        col.append(feature_id)
col = np.array(col)
row=[]
for item_id in pad_items_ids:
    for feature_id in range(len(feature_ids)):
        row.append(item_id)
row=np.array(row)
#print(col)
#print(row)
#print(pad_items_ids.size)
#print(feature_ids)
#print(data)
data = np.zeros((row.size),dtype=int)
col = np.transpose(col)
print(col.shape)
print(row.shape)
print(data.shape)
#padICM = sps.coo_matrix((data,(row,col)),shape=(row.size, col.size))
#paddedICM = sps.vstack(ICM, padICM)
#sps.vstack(URM_train, paddedICM.T)
shape = (len(row), len(col))

# Create indices for users and items
#row_cat = CategoricalDtype(categories=sorted(row), ordered=True)
#col_cat = CategoricalDtype(categories=sorted(col), ordered=True)
#row_index = dataframe[row_name].astype(row_cat).cat.codes
#col_index = dataframe[col_name].astype(col_cat).cat.codes

# Conversion via COO matrix
coo = sps.coo_matrix(
    (data, (row.values, col.values)), shape=shape)
csr = coo.tocsr()
csr

(24385,)
(24385,)
(24385,)


AttributeError: 'numpy.ndarray' object has no attribute 'values'